<a href="https://colab.research.google.com/github/ClaudiaMarano/Anomaly-Detection-and-Prediction/blob/main/network_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Physical Anomaly Detection

## Import Librerie

In [53]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
from tensorflow.keras.callbacks import Callback
from keras.optimizers import Adam
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.layers import LeakyReLU
from keras.layers import Dropout

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


## Caricamento e Preprocessing
Il dataset viene suddiviso in intervalli di un minuto in modo da addestrare la rete su intervalli di campioni senza anomalie, in modo da poter riconoscere in fase di test quando un intervallo contiene invece un'anomalia.


In [58]:
def preprocess_by_minute(path):
    # Caricamento del dataset
    df = pd.read_csv(path, encoding="utf-8")

    # Assicuriamoci che 'Time' sia in formato datetime
    df['Time'] = pd.to_datetime(df['Time'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')

    # Rimuovi righe con valori di data non validi
    df = df.dropna(subset=['Time'])

    # Ordinare i dati per timestamp
    df = df.sort_values(by='Time')

    print("Start Time: ", df['Time'].iloc[0])
    print("Finish Time: ", df['Time'].iloc[-1])

    # Raggruppa i dati in intervalli di 1 minuto
    grouped = df.groupby(pd.Grouper(key='Time', freq='1min'))

    # Creare i segmenti (ogni gruppo corrisponde a 1 minuto)
    segments = []
    for _, group in grouped:
        if not group.empty:
          segment = group.drop(columns=['Time', 'label', 'label_n']).reset_index(drop=True)
          # segment.to_csv(f'segment_{len(segments)}.csv', index=False)
          segments.append(segment)

    # Restituisci i segmenti
    return segments

In [57]:
def preprocess_network_data(path):
    # Caricamento del dataset
    df = pd.read_csv(path, encoding="utf-8")

    # Assicuriamoci che 'Time' sia in formato datetime
    df['Time'] = pd.to_datetime(df['Time'], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')

    # Rimuovi righe con valori di data non validi
    df = df.dropna(subset=['Time'])

    # Ordinare i dati per timestamp
    df_normal = df.sort_values(by='Time')

    # Divisione in intervalli di un minuto
    segments = []
    window_duration = pd.Timedelta(minutes=1)
    start_time = df_normal['Time'].iloc[0]

    print("Start Time: ", df_normal['Time'].iloc[0])
    print("Finish Time: ", df_normal['Time'].iloc[-1])

    while start_time < df_normal['Time'].iloc[-1]:
        end_time = start_time + window_duration
        segment = df_normal[(df_normal['Time'] >= start_time) & (df_normal['Time'] < end_time)]
        if len(segment) > 0:
            segments.append(segment.drop(columns=['Time', 'label', 'label_n']).reset_index(drop=True))
        start_time = end_time

    # Preprocessing delle feature
    # Separiamo colonne categoriali e numeriche
    categorical_columns = ['mac_s', 'mac_d', 'ip_s', 'ip_d', 'proto', 'flags', 'modbus_fn']
    numerical_columns = ['sport', 'dport', 'size', 'n_pkt_src', 'n_pkt_dst']

    # Creiamo un trasformatore combinato
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns),
            ('num', MinMaxScaler(), numerical_columns)
        ]
    )

    # Normalizzazione e codifica di ogni segmento
    processed_segments = []
    for segment in segments:
        processed = preprocessor.fit_transform(segment)
        processed_segments.append(processed)

    # Ritorniamo i segmenti processati (normalizzati e codificati)
    return processed_segments

## Funzione per il Training

In [ ]:
# 3.
def building_and_training(segments_scaled_split):
    # Callback per visualizzare statistiche al termine dell'addestramento
    class TrainingSummary(Callback):
        def on_train_end(self, logs=None):
            print("\n--- Statistiche Finali ---")
            print(f"Loss finale su training set: {logs['loss']:.4f}")
            if 'val_loss' in logs:
                print(f"Loss finale su validation set: {logs['val_loss']:.4f}")

    # Definisco l'input shape
    input_shape = segments_scaled_split[0].shape  # Forma di un segmento

    # Definisco la struttura dell'autoencoder
    input_layer = Input(shape=input_shape)
    x = Flatten()(input_layer)
    x = Dense(256)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)  # Dropout 30%
    x = Dense(128)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)  # Dropout 30%
    encoded = Dense(64)(x)
    encoded = LeakyReLU(alpha=0.1)(encoded)
    x = Dense(128)(encoded)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)  # Dropout 30%
    x = Dense(256)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)  # Dropout 30%
    x = Dense(np.prod(input_shape), activation="sigmoid")(x)
    x = BatchNormalization()(x)
    decoded = Reshape(input_shape)(x)

    # Creo il Modello
    autoencoder = Model(inputs=input_layer, outputs=decoded)
    autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss="mse")

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    # Addestramento dell'autoencoder sui dati "normali", senza anomalie
    history = autoencoder.fit(
        np.array(segments_scaled_split), np.array(segments_scaled_split),
        epochs=500, batch_size=16, shuffle=True, validation_split=0.1,
        callbacks=[TrainingSummary(), early_stopping]
    )

    # Stampo statistiche finali direttamente dal dizionario `history.history`

    print("\n--- Risultati Finali ---")
    print(f"Training Loss: {history.history['loss'][-1]:.4f}")
    print(f"Validation Loss: {history.history['val_loss'][-1]:.4f}")

    return autoencoder


## Calcolo dell'Errore di Ricostruzione

In [ ]:
# 4.
def get_rebuilding_error(autoencoder, segments_scaled_split):
    # Ricostruzione dei dati di addestramento
    reconstructed_train = autoencoder.predict(np.array(segments_scaled_split))
    mse_train = np.mean(np.power(segments_scaled_split - reconstructed_train, 2), axis=(1, 2))

    # Imposto il threshold al 95° percentile dell'errore
    threshold = np.percentile(mse_train, 95)
    print("Soglia di errore di ricostruzione:", threshold)

    return threshold

## Test del modello

In [ ]:
# 5.
def testing(path, scaler, autoencoder, threshold):
    # Carico il CSV con diversa codifica
    df_anomalous = pd.read_csv(path, sep='\t', encoding='utf-16')
    df_anomalous = df_anomalous.drop(columns=["Label"])

    df_anomalous['Time'] = pd.to_datetime(df_anomalous['Time'])
    df_anomalous = df_anomalous.sort_values(by='Time')

    # Segmentazione del file di test in blocchi di tot minuti
    test_segments = []
    start_time = df_anomalous['Time'].iloc[0]

    # Definisco la finestra di tot minuti
    window_duration = pd.Timedelta(minutes=1)

    while start_time < df_anomalous['Time'].iloc[-1]:
        end_time = start_time + window_duration
        segment = df_anomalous[(df_anomalous['Time'] >= start_time) & (df_anomalous['Time'] < end_time)]
        if len(segment) > 0:
            test_segments.append(segment.drop(columns=['Time']).values)
        start_time = end_time

    # Normalizzazione dei segmenti di test usando lo scaler già addestrato
    test_segments_scaled = [scaler.transform(segment) for segment in test_segments]

    # Mantengo solo i segmenti che hanno lunghezza pari a 60 o 120 in base a se divido in intervalli di tot minuti.
    uniform_segments = [segment for segment in test_segments_scaled if len(segment) == 60]

    for seg in uniform_segments:
        print(seg)

    # Ricostruzione e calcolo dell'errore per ogni segmento di test
    reconstructed_test = autoencoder.predict(np.array(uniform_segments))
    mse_test = np.mean(np.power(uniform_segments - reconstructed_test, 2), axis=(1, 2))

    # Identificazione delle anomalie nei blocchi di test
    anomalies = mse_test > threshold
    print("Numero di blocchi anomali rilevati:", np.sum(anomalies))
    print(f"Blocchi anomali: {anomalies}")

## Esecuzione del processo

In [59]:
segments = preprocess_by_minute("normal.csv")
print("Numero di segmenti:", len(segments))

Start Time:  2021-04-09 11:30:52.716203
Finish Time:  2021-04-09 12:07:36.011694
Numero di segmenti: 38


In [60]:
segments_2 = preprocess_network_data("normal.csv")
print("Numero di segmenti:", len(segments_2))

Start Time:  2021-04-09 11:30:52.716203
Finish Time:  2021-04-09 12:07:36.011694
Numero di segmenti: 37


In [ ]:
autoencoder = building_and_training(segments)

In [ ]:
threshols = get_rebuilding_error(autoencoder, segments_scaled_split)

In [ ]:
testing("attack_1.csv", scaler, autoencoder, threshols)